In [1]:
import re
import operator
from math import floor
from heapq import heappop, heappush

In [2]:
with open('../data/2024/day17.txt') as f:
    data = f.read()

In [3]:
# Parse input
a,b,c,program = re.sub(r'[a-z ]+\: (.*)', '\\1', data, 0, re.IGNORECASE).replace('\n\n','\n').split('\n')
program = list(map(int, program.split(',')))
registers = {'a': int(a), 'b': int(b), 'c': int(c)}
original_program = list(program)
original_registers = dict(registers)

In [4]:
def combo(operand):
    if operand in [0,1,2,3]: return operand
    if operand in [4,5,6]: return registers[['a','b','c'][operand-4]]
    return None

def instruction(opcode, operand) -> tuple: # jmp, out
    if 0 == opcode: # adv
        registers['a'] = floor(registers['a'] / pow(2, combo(operand)))
    elif 1 == opcode: # bxl
        registers['b'] = operator.xor(registers['b'], operand)
    elif 2 == opcode: # bst
        registers['b'] = combo(operand) % 8
    elif 3 == opcode: #jnz
        if 0 != registers['a']: return operand, None
    elif 4 == opcode: #bxc
        registers['b'] = operator.xor(registers['b'], registers['c'])
    elif 5 == opcode: #out
        return None, combo(operand) % 8
    elif 6 == opcode: #bdv
        registers['b'] = floor(registers['a'] / pow(2, combo(operand)))
    elif 7 == opcode: #cdv
        registers['c'] = floor(registers['a'] / pow(2, combo(operand)))
    return None, None

def execute(commands) -> str:
    pointer, jmp, output = 0, None, []

    while True:
        try:
            jmp, out = instruction(*commands[pointer:pointer+2])
            pointer = jmp if jmp is not None else (pointer + 2)
            if out is not None: output.append(out)
        except IndexError: break
        except TypeError: break
    return output

In [5]:
# Part 1
print("Part 1:", ','.join([str(o) for o in execute(program)]))

Part 1: 3,1,5,3,7,4,2,7,5


In [6]:
# Part 2
# Register `a` starts at pow(8,k) where k=(desired length-1)
# Digit sequence: 4,5,7,1,0,3,2 per pow(8,k) with k-1 moving right-to-left
program = list(original_program)
l = len(original_program)

state = (0, [-1 for _ in range(l)], l-1)
queue = [(0, *state)] # Heap to prioritize the longest valid candidates

# For each candidate sequence
while queue:
    # Candidate register a value, candidate program, k (unsolved index from right)
    _, a, candidate, k = heappop(queue)

    if k == -1: # If we've predicted all the digits
        if candidate == original_program: # And we have a match
            print("Part 2:", a)
            break
        else: continue

    # Try a multiple of 0-7 for each position to line up the digits
    for n in range(8):
        new_a = a + pow(8,k) * n

        # Run the computer to create a new candidate program
        registers = dict(original_registers)
        registers['a'] = new_a
        candidate = execute(program)

        # If the candidate tail matches the actual output tail, move left a digit
        if candidate[-(16 - k):] == original_program[-(16 - k):]:
            heappush(queue, (15-k, new_a, candidate, k - 1))

Part 2: 190593310997519
